In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**load old Austrian happiness scores as csv file into kaggle and store contents in a variable dataframe **

In [ ]:
dataframe = pd.read_csv("/kaggle/input/austrian-happieness-old-scores/the Old_happieness_scores-2017-2021.csv")


In [ ]:
# print first 
dataframe.head(5)

drop the country column 

In [ ]:
df = dataframe
df.dtypes

describe the data and explore the mean , saddest year was 2017 and happiest year is 2020 

In [ ]:
df.describe()
df.pivot_table(values=["Happiness_Score"], index=["year"])

**import Plot library for visulaization**

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot([2017,2018,2019,2020,2021],[7.0060,7.1390,7.2460,7.2942,7.2680], "y")
plt.axis([2017,2021,7,7.3])
plt.xlabel("year")
plt.ylabel("Happiness.Score")
plt.title("Austrian Happiness score for last 5 years")
plt.text(2020.3,7.28, "Covid Crisis")


> The falling happiness started in March 2020

In [ ]:
plt.savefig("Austrian Happiness score for last 5 years.png")

****import scikit-learn library to start the machine learning predection****

In [ ]:
from sklearn.preprocessing import LabelEncoder

****encode object columns ****

In [ ]:
columns_2_encode = ["Country","Health_Life_Expectancy", "Happiness_Score", "Freedom", "Generosity", "Trust_Government_Corruption"]

labelEncoder = LabelEncoder()

for i in columns_2_encode:
    df[i] = labelEncoder.fit_transform(df[i])

In [ ]:
df.head(5)

In [ ]:
df.dtypes

**for the categorized data we need a classification algorithm  **


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn import metrics


****for accuracy and validation we create classification model and Data training 
K fold data validation ****

In [ ]:
def classification_model(model, data, predictors, outcome, kfoldnumber):
    ## fit data 
    model.fit(data[predictors], data[outcome])
    ## predecit train data 
    predictvalues =model.predict(data[predictors])
    ##
    print("PredictValues")
    print(predictvalues)
    print(data[outcome])
    ## accuracy 
    accuracy = metrics.accuracy_score(predictvalues, data[outcome])
    print("accuracy: %s" % "{0:.3%}".format(accuracy))
    ## k fold accros validation 
    kfold = KFold(n_splits=kfoldnumber)
    error = []
    ##
    for train, test in kfold.split(data):
        print("----run----")
        print("traindata")
        print("train")
        print("testdata")
        print(test)
        ##
        ## filter data 
        train_data = data[predictors].iloc[train,:]
        train_target = data[outcome].iloc[train]
        ##
        print("TrainData")
        print(train_data)
        print("TrainTarget")
        print(train_target)
        ## fit data 
        model.fit(train_data, train_target)
        ##
        error.append(model.score(data[predictors].iloc[test,:], data[outcome].iloc[test]))
        print("Cross Validation Score: %s" % "{0:.3%}".format(np.mean(error)))
        ###
        model.fit(data[predictors]. data[outcome])
        
    


**Logistic regression**

In [ ]:
outcome_var = "Happiness_Score"
predictor_var = ["Health_Life_Expectancy", "Freedom", "Generosity", "Trust_Government_Corruption"]
model = LogisticRegression(solver="lbfgs")
##
classification_model(model, df, predictor_var, outcome_var, 2)


**Random forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
##
model = RandomForestClassifier(n_estimators=5)
#
outcome_var = "Happiness_Score"
predictor_var = ["Health_Life_Expectancy", "Freedom", "Generosity", "Trust_Government_Corruption"]
##
classification_model(model, df, predictor_var, outcome_var, 2)


**feature importance**

In [ ]:
feature_importance = pd.Series(model.feature_importances_, index=predictor_var).sort_values(ascending=False)
print(feature_importance)                                                      

In [ ]:
import matplotlib.pyplot as plt
plt.title("Happiness score features importance")
Happiness_Score = [0.300794,0.328571,0.172222,0.198413]
factors = ["Freedom","Health_Life_Expectancy","Trust_Government_Corruption","Generosity"]
exp = [0.1,0.2,0,0]
plt.pie(Happiness_Score, labels = factors, explode = exp, autopct = "%2.1f%%")
plt.show()
plt.savefig("Happiness score features importance.png")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
##
model = RandomForestClassifier(n_estimators=100)
#
outcome_var = "Happiness_Score"
predictor_var = ["Health_Life_Expectancy", "Freedom", "Generosity", "Trust_Government_Corruption"]
##
classification_model(model, df, predictor_var, outcome_var, 2)

Print decision tree

In [ ]:
import graphviz
from sklearn.tree import export_graphviz

In [ ]:
dot_data = export_graphviz(model, out_file=None, feature_names=predictor_var, filled=True, rounded=True, special_characters=True)
graph=graphviz.Source(dot_data)
graph